<a href="https://colab.research.google.com/github/vggls/medical_xai/blob/main/experiments/non_medical_datasets_haas/Imagenette_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports**

In [ ]:
pip install grad-cam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=38261 sha256=3ec94f6ef143b7ee63cd6ba25677de8cf3af8ccf74077c5bef8ebc2005e93fbf
  Stored in directory: /root/.cache/pip/wheels/53/02/43/1f75726b5c28459596067ad91e36951463c01273eef661f09f
Successfully built grad-cam


In [ ]:
from torchvision import transforms, datasets
import torch
import matplotlib.pyplot as plt
from torch import nn
import torch.optim as optim
from torch.utils.data import random_split
from pytorch_grad_cam import HiResCAM
import pickle
import os
from torch.utils.data import DataLoader

In [ ]:
from imagenette import create_datasets, summaries
from training_loop import Train
from train_model import fit
from vgg19 import VGG19
from testing_report import Test_Report
from haas import Haas, HA_image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Load Data**

In [ ]:
# download dataset from https://github.com/fastai/imagenette
# convert tgz file to zip
# upload zip file on google drive
# unzip as per below
!unzip "./drive/My Drive/Datasets/Imagenette/imagenette2.zip"

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
  inflating: imagenette2/train/n03445777/n03445777_8446.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8448.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8474.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8477.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8484.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8544.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8546.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8599.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8616.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8624.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8628.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_866.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8663.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8677.JPEG  
  inflating: imagenette2/train/n03445777/n03445777_8678.JPEG  
 

In [ ]:
folders = {'tench': 'n01440764',
          'English springer': 'n02102040',
          'cassette player': 'n02979186',
          'chain saw': 'n03000684',
          'church': 'n03028079',
          'French horn': 'n03394916',
          'garbage truck': 'n03417042',
          'gas pump': 'n03425413',
          'golf ball': 'n03445777',
          'parachute': 'n03888257'}

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
train_dataset, validation_dataset, test_dataset = create_datasets(train_transforms, test_transforms)

In [ ]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(5676, 3793, 3925)

In [ ]:
summaries(True)

## **VGG19 (loop1, lr=1e-06, batch_size=32, 2 models)**

**Internet connection interupted. Rest models at 1e-06 learning rate are calculated in loop2**

In [ ]:
drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/Imagenette/'

count = 0

balanced_accuracies, weighted_f1_scores, mean_roc_auc_scores = [], [], []
haas_scores = []

#weights = class_weights('./training_dataset', train_dataset)

for lr in [0.000001]:
    for bs in [32, 64, 128, 256]:
        for scheduler in ['yes', 'no']:

            count += 1

            vgg19 = VGG19(num_classes = 10,
                      trainable_feature_layers = None)

            loss_fct = torch.nn.CrossEntropyLoss()

            optimizer = optim.Adam(vgg19.model.parameters(), lr=lr)

            if scheduler == 'yes':
              sched = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
            else:
              sched = None

            print('\033[1m' + 'Model no.{} - lr:{} - bs:{} - scheduler:{}'.format(count, lr, bs, scheduler) + '\033[0m')

            training_dict, validation_dict = fit(train_dataset=train_dataset,
                                                validation_dataset=validation_dataset,
                                                batch_size=bs,
                                                model=vgg19,
                                                loss_fct=loss_fct,
                                                optimizer=optimizer,
                                                scheduler=sched,
                                                epochs=8,
                                                patience=None,
                                                no_of_classes=10,
                                                labels_of_normal_classes=None)

            model = torch.load('model_epoch8.pt')
            model = model.cuda()
            model = model.eval()

            # MODEL CLASSIFICATION METRICS
            test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)
            classes = folders.keys()
            test_report = Test_Report(test_loader, model, classes)

            balanced_accuracies.append(test_report.balanced_accuracy())

            weighted_f1_scores.append(test_report.f1('weighted'))

            model_roc_auc_scores = test_report.roc_curve_and_scores(False)
            mean_roc_auc_scores.append(sum(model_roc_auc_scores)/len(model_roc_auc_scores))

            # MODEL HAAS SCORE
            for child in model.features:
                for param in child.parameters():
                    param.requires_grad = True

            print('HiResCAM - HAAS calculation')
            hirescam_instance = HiResCAM(model=model, target_layers=[model.features[-1]], use_cuda=True)
            hirescam_score, _, _, _ = Haas(test_dataset, model, hirescam_instance)
            haas_scores.append(hirescam_score)
            print('HAAS score:', hirescam_score)

            if hirescam_score>1:
              torch.save(model, drive_path + 'imagenette_vgg19_no{}.pt'.format(count))
              with open(drive_path + 'imagenette_vgg19_no{}_training_dict.pickle'.format(count), 'wb') as f:
                pickle.dump(training_dict, f)
              with open(drive_path + 'imagenette_vgg19_no{}_validation_dict.pickle'.format(count), 'wb') as f:
                pickle.dump(validation_dict, f)

            os.remove('model_epoch8.pt')

            print('*'*100)
            print('\n')

Model no.1 - lr:1e-06 - bs:32 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  lr value 1e-06
  Loss=2.0268
  Accuracy=0.39
->Validation phase
  Loss=1.0178
  Accuracy=0.67
-----------------------------------
Epoch 2/8
->Training phase
  lr value 1e-06
  Loss=0.7888
  Accuracy=0.75
->Validation phase
  Loss=0.6134
  Accuracy=0.81
-----------------------------------
Epoch 3/8
->Training phase
  lr value 5e-07
  Loss=0.5423
  Accuracy=0.84
->Validation phase
  Loss=0.5085
  Accuracy=0.85
-----------------------------------
Epoch 4/8
->Training phase
  lr value 5e-07
  Loss=0.4565
  Accuracy=0.87
->Validation phase
  Loss=0.4410
  Accuracy=0.87
-----------------------------------
Epoch 5/8
->Training phase
  lr value 2.5e-07
  Loss=0.4014
  Accuracy=0.89
->Validation phase
  Loss=0.4181
  Accuracy=0.88
-----------------------------------
Epoch 6/8
->Training phase
  lr value 2.5e-07
  Loss=0.3759
  Accuracy=0.89
->Validati

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model no.2 - lr:1e-06 - bs:32 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  Loss=1.7269
  Accuracy=0.46
->Validation phase
  Loss=0.9007
  Accuracy=0.71
-----------------------------------
Epoch 2/8
->Training phase
  Loss=0.6793
  Accuracy=0.79
->Validation phase
  Loss=0.5586
  Accuracy=0.83
-----------------------------------
Epoch 3/8
->Training phase
  Loss=0.4487
  Accuracy=0.87
->Validation phase
  Loss=0.4273
  Accuracy=0.87
-----------------------------------
Epoch 4/8
->Training phase
  Loss=0.3456
  Accuracy=0.9
->Validation phase
  Loss=0.3498
  Accuracy=0.9
-----------------------------------
Epoch 5/8
->Training phase
  Loss=0.2813
  Accuracy=0.92
->Validation phase
  Loss=0.3047
  Accuracy=0.91
-----------------------------------
Epoch 6/8
->Training phase
  Loss=0.2384
  Accuracy=0.93
->Validation phase
  Loss=0.2871
  Accuracy=0.91
-----------------------------------
Epoch 7/8
->Training phase
  Loss=

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model no.3 - lr:1e-06 - bs:64 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  lr value 1e-06
  Loss=2.4277
  Accuracy=0.28
->Validation phase
  Loss=1.4383
  Accuracy=0.52
-----------------------------------
Epoch 2/8
->Training phase
  lr value 1e-06
  Loss=1.0936
  Accuracy=0.65
->Validation phase
  Loss=0.8396
  Accuracy=0.74
-----------------------------------
Epoch 3/8
->Training phase
  lr value 5e-07
  Loss=0.7483
  Accuracy=0.77
->Validation phase
  Loss=0.7068
  Accuracy=0.79
-----------------------------------
Epoch 4/8
->Training phase
  lr value 5e-07
  Loss=0.6297
  Accuracy=0.81
->Validation phase
  Loss=0.6037
  Accuracy=0.82
-----------------------------------
Epoch 5/8
->Training phase
  lr value 2.5e-07
  Loss=0.5570
  Accuracy=0.84
->Validation phase
  Loss=0.5692
  Accuracy=0.83
-----------------------------------
Epoch 6/8
->Training phase
  lr value 2.5e-07
  Loss=0.5208
  Accuracy=0.85
->Validati

In [ ]:
with open(drive_path + 'imagenette_haas_scores_loop1.pickle', 'wb') as f: pickle.dump(haas_scores, f)
with open(drive_path + 'imagenette_balanced_accuracies_loop1.pickle', 'wb') as f: pickle.dump(balanced_accuracies, f)
with open(drive_path + 'imagenette_weighted_f1_scores_loop1.pickle', 'wb') as f: pickle.dump(weighted_f1_scores, f)
with open(drive_path + 'imagenette_mean_roc_auc_scores_loop1.pickle', 'wb') as f: pickle.dump(mean_roc_auc_scores, f)

## **VGG19 (loop2, lr=1e-06, batch size=[64,128,256], 6 models)**

**Continuation of loop1**

In [ ]:
drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/Imagenette/'

count = 2

balanced_accuracies, weighted_f1_scores, mean_roc_auc_scores = [], [], []
haas_scores = []

for lr in [0.000001]: #SAME RATE AS LOOP 1
    for bs in [64, 128, 256]: #RECALL THAT BATCH SIZE 32 WAS CALCULATED IN LOOP1
        for scheduler in ['no', 'yes']:

            count += 1

            vgg19 = VGG19(num_classes = 10,
                      trainable_feature_layers = None)

            loss_fct = torch.nn.CrossEntropyLoss()

            optimizer = optim.Adam(vgg19.model.parameters(), lr=lr)

            if scheduler == 'yes':
              sched = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
            else:
              sched = None

            print('\033[1m' + 'Model no.{} - lr:{} - bs:{} - scheduler:{}'.format(count, lr, bs, scheduler) + '\033[0m')

            training_dict, validation_dict = fit(train_dataset=train_dataset,
                                                validation_dataset=validation_dataset,
                                                batch_size=bs,
                                                model=vgg19,
                                                loss_fct=loss_fct,
                                                optimizer=optimizer,
                                                scheduler=sched,
                                                epochs=8,
                                                patience=None,
                                                no_of_classes=10,
                                                labels_of_normal_classes=None)

            model = torch.load('model_epoch8.pt')
            model = model.cuda()
            model = model.eval()

            # MODEL CLASSIFICATION METRICS
            test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)
            classes = folders.keys()
            test_report = Test_Report(test_loader, model, classes)

            balanced_accuracies.append(test_report.balanced_accuracy())

            weighted_f1_scores.append(test_report.f1('weighted'))

            model_roc_auc_scores = test_report.roc_curve_and_scores(False)
            mean_roc_auc_scores.append(sum(model_roc_auc_scores)/len(model_roc_auc_scores))

            # MODEL HAAS SCORE
            for child in model.features:
                for param in child.parameters():
                    param.requires_grad = True

            print('HiResCAM - HAAS calculation')
            hirescam_instance = HiResCAM(model=model, target_layers=[model.features[-1]], use_cuda=True)
            hirescam_score, _, _, _ = Haas(test_dataset, model, hirescam_instance)
            haas_scores.append(hirescam_score)
            print('HAAS score:', hirescam_score)

            if hirescam_score>1:
              torch.save(model, drive_path + 'imagenette_vgg19_no{}.pt'.format(count))
              with open(drive_path + 'imagenette_vgg19_no{}_training_dict.pickle'.format(count), 'wb') as f:
                pickle.dump(training_dict, f)
              with open(drive_path + 'imagenette_vgg19_no{}_validation_dict.pickle'.format(count), 'wb') as f:
                pickle.dump(validation_dict, f)

            os.remove('model_epoch8.pt')

            print('*'*100)
            print('\n')

Model no.3 - lr:1e-06 - bs:64 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  Loss=2.5871
  Accuracy=0.27
->Validation phase
  Loss=1.5511
  Accuracy=0.49
-----------------------------------
Epoch 2/8
->Training phase
  Loss=1.1780
  Accuracy=0.62
->Validation phase
  Loss=0.8986
  Accuracy=0.71
-----------------------------------
Epoch 3/8
->Training phase
  Loss=0.7525
  Accuracy=0.76
->Validation phase
  Loss=0.6487
  Accuracy=0.81
-----------------------------------
Epoch 4/8
->Training phase
  Loss=0.5632
  Accuracy=0.83
->Validation phase
  Loss=0.5255
  Accuracy=0.84
-----------------------------------
Epoch 5/8
->Training phase
  Loss=0.4554
  Accuracy=0.87
->Validation phase
  Loss=0.4473
  Accuracy=0.86
-----------------------------------
Epoch 6/8
->Training phase
  Loss=0.3838
  Accuracy=0.89
->Validation phase
  Loss=0.4015
  Accuracy=0.88
-----------------------------------
Epoch 7/8
->Training phase
  Los

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.4 - lr:1e-06 - bs:64 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  lr value 1e-06
  Loss=2.2966
  Accuracy=0.31
->Validation phase
  Loss=1.4326
  Accuracy=0.52
-----------------------------------
Epoch 2/8
->Training phase
  lr value 1e-06
  Loss=1.0648
  Accuracy=0.66
->Validation phase
  Loss=0.8339
  Accuracy=0.74
-----------------------------------
Epoch 3/8
->Training phase
  lr value 5e-07
  Loss=0.7387
  Accuracy=0.77
->Validation phase
  Loss=0.6961
  Accuracy=0.79
-----------------------------------
Epoch 4/8
->Training phase
  lr value 5e-07
  Loss=0.6240
  Accuracy=0.81
->Validation phase
  Loss=0.6015
  Accuracy=0.82
-----------------------------------
Epoch 5/8
->Training phase
  lr value 2.5e-07
  Loss=0.5522
  Accuracy=0.83
->Validation phase
  Loss=0.5593
  Accuracy=0.84
-----------------------------------
Epoch 6/8
->Training phase
  lr value 2.5e-07
  Loss=0.5159
  Accuracy=0.84
->Validati

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.5 - lr:1e-06 - bs:128 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  Loss=3.1826
  Accuracy=0.16
->Validation phase
  Loss=2.2224
  Accuracy=0.29
-----------------------------------
Epoch 2/8
->Training phase
  Loss=1.7663
  Accuracy=0.4
->Validation phase
  Loss=1.3720
  Accuracy=0.53
-----------------------------------
Epoch 3/8
->Training phase
  Loss=1.1465
  Accuracy=0.62
->Validation phase
  Loss=0.9743
  Accuracy=0.68
-----------------------------------
Epoch 4/8
->Training phase
  Loss=0.8443
  Accuracy=0.73
->Validation phase
  Loss=0.7689
  Accuracy=0.76
-----------------------------------
Epoch 5/8
->Training phase
  Loss=0.6732
  Accuracy=0.8
->Validation phase
  Loss=0.6447
  Accuracy=0.8
-----------------------------------
Epoch 6/8
->Training phase
  Loss=0.5661
  Accuracy=0.84
->Validation phase
  Loss=0.5691
  Accuracy=0.83
-----------------------------------
Epoch 7/8
->Training phase
  Loss=

...

Model no.6 - lr:1e-06 - bs:128 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  lr value 1e-06
  Loss=2.5862
  Accuracy=0.22
->Validation phase
  Loss=1.8659
  Accuracy=0.38
-----------------------------------
Epoch 2/8
->Training phase
  lr value 1e-06
  Loss=1.4925
  Accuracy=0.51
->Validation phase
  Loss=1.1759
  Accuracy=0.61
-----------------------------------
Epoch 3/8
->Training phase
  lr value 5e-07
  Loss=1.0795
  Accuracy=0.65
->Validation phase
  Loss=0.9859
  Accuracy=0.68
-----------------------------------
Epoch 4/8
->Training phase
  lr value 5e-07
  Loss=0.9157
  Accuracy=0.71
->Validation phase
  Loss=0.8492
  Accuracy=0.73
-----------------------------------
Epoch 5/8
->Training phase
  lr value 2.5e-07
  Loss=0.8153
  Accuracy=0.75
->Validation phase
  Loss=0.7944
  Accuracy=0.75
-----------------------------------
Epoch 6/8
->Training phase
  lr value 2.5e-07
  Loss=0.7604
  Accuracy=0.77
->Validat

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.7 - lr:1e-06 - bs:256 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  Loss=3.0146
  Accuracy=0.16
->Validation phase
  Loss=2.4131
  Accuracy=0.26
-----------------------------------
Epoch 2/8
->Training phase
  Loss=2.0929
  Accuracy=0.34
->Validation phase
  Loss=1.7305
  Accuracy=0.44
-----------------------------------
Epoch 3/8
->Training phase
  Loss=1.5227
  Accuracy=0.5
->Validation phase
  Loss=1.2987
  Accuracy=0.59
-----------------------------------
Epoch 4/8
->Training phase
  Loss=1.1718
  Accuracy=0.62
->Validation phase
  Loss=1.0326
  Accuracy=0.69
-----------------------------------
Epoch 5/8
->Training phase
  Loss=0.9640
  Accuracy=0.7
->Validation phase
  Loss=0.8672
  Accuracy=0.74
-----------------------------------
Epoch 6/8
->Training phase
  Loss=0.8228
  Accuracy=0.74
->Validation phase
  Loss=0.7544
  Accuracy=0.77
-----------------------------------
Epoch 7/8
->Training phase
  Loss

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.8 - lr:1e-06 - bs:256 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  lr value 1e-06
  Loss=3.6962
  Accuracy=0.12
->Validation phase
  Loss=2.8144
  Accuracy=0.2
-----------------------------------
Epoch 2/8
->Training phase
  lr value 1e-06
  Loss=2.4584
  Accuracy=0.24
->Validation phase
  Loss=2.0734
  Accuracy=0.32
-----------------------------------
Epoch 3/8
->Training phase
  lr value 5e-07
  Loss=1.9534
  Accuracy=0.36
->Validation phase
  Loss=1.8087
  Accuracy=0.4
-----------------------------------
Epoch 4/8
->Training phase
  lr value 5e-07
  Loss=1.7143
  Accuracy=0.44
->Validation phase
  Loss=1.5911
  Accuracy=0.48
-----------------------------------
Epoch 5/8
->Training phase
  lr value 2.5e-07
  Loss=1.5473
  Accuracy=0.49
->Validation phase
  Loss=1.4978
  Accuracy=0.51
-----------------------------------
Epoch 6/8
->Training phase
  lr value 2.5e-07
  Loss=1.4564
  Accuracy=0.52
->Validatio

In [ ]:
with open(drive_path + 'imagenette_haas_scores_loop2.pickle', 'wb') as f: pickle.dump(haas_scores, f)
with open(drive_path + 'imagenette_balanced_accuracies_loop2.pickle', 'wb') as f: pickle.dump(balanced_accuracies, f)
with open(drive_path + 'imagenette_weighted_f1_scores_loop2.pickle', 'wb') as f: pickle.dump(weighted_f1_scores, f)
with open(drive_path + 'imagenette_mean_roc_auc_scores_loop2.pickle', 'wb') as f: pickle.dump(mean_roc_auc_scores, f)

## **VGG19 (loop3, lr=1e-04, 8 models)**

In [ ]:
drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/Imagenette/'

count = 8

balanced_accuracies, weighted_f1_scores, mean_roc_auc_scores = [], [], []
haas_scores = []

for lr in [0.0001]:
    for bs in [32, 64, 128, 256]:
        for scheduler in ['no', 'yes']:

            count += 1

            vgg19 = VGG19(num_classes = 10,
                      trainable_feature_layers = None)

            loss_fct = torch.nn.CrossEntropyLoss()

            optimizer = optim.Adam(vgg19.model.parameters(), lr=lr)

            if scheduler == 'yes':
              sched = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
            else:
              sched = None

            print('\033[1m' + 'Model no.{} - lr:{} - bs:{} - scheduler:{}'.format(count, lr, bs, scheduler) + '\033[0m')

            training_dict, validation_dict = fit(train_dataset=train_dataset,
                                                validation_dataset=validation_dataset,
                                                batch_size=bs,
                                                model=vgg19,
                                                loss_fct=loss_fct,
                                                optimizer=optimizer,
                                                scheduler=sched,
                                                epochs=8,
                                                patience=None,
                                                no_of_classes=10,
                                                labels_of_normal_classes=None)

            model = torch.load('model_epoch8.pt')
            model = model.cuda()
            model = model.eval()

            # MODEL CLASSIFICATION METRICS
            test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)
            classes = folders.keys()
            test_report = Test_Report(test_loader, model, classes)

            balanced_accuracies.append(test_report.balanced_accuracy())

            weighted_f1_scores.append(test_report.f1('weighted'))

            model_roc_auc_scores = test_report.roc_curve_and_scores(False)
            mean_roc_auc_scores.append(sum(model_roc_auc_scores)/len(model_roc_auc_scores))

            # MODEL HAAS SCORE
            for child in model.features:
                for param in child.parameters():
                    param.requires_grad = True

            print('HiResCAM - HAAS calculation')
            hirescam_instance = HiResCAM(model=model, target_layers=[model.features[-1]], use_cuda=True)
            hirescam_score, _, _, _ = Haas(test_dataset, model, hirescam_instance)
            haas_scores.append(hirescam_score)
            print('HAAS score:', hirescam_score)

            if hirescam_score>1:
              torch.save(model, drive_path + 'imagenette_vgg19_no{}.pt'.format(count))
              with open(drive_path + 'imagenette_vgg19_no{}_training_dict.pickle'.format(count), 'wb') as f:
                pickle.dump(training_dict, f)
              with open(drive_path + 'imagenette_vgg19_no{}_validation_dict.pickle'.format(count), 'wb') as f:
                pickle.dump(validation_dict, f)

            os.remove('model_epoch8.pt')

            print('*'*100)
            print('\n')

Model no.9 - lr:0.0001 - bs:32 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  Loss=3.6002
  Accuracy=0.82
->Validation phase
  Loss=0.3891
  Accuracy=0.94
-----------------------------------
Epoch 2/8
->Training phase
  Loss=0.4625
  Accuracy=0.93
->Validation phase
  Loss=0.8137
  Accuracy=0.9
-----------------------------------
Epoch 3/8
->Training phase
  Loss=0.5802
  Accuracy=0.93
->Validation phase
  Loss=0.8300
  Accuracy=0.93
-----------------------------------
Epoch 4/8
->Training phase
  Loss=0.6726
  Accuracy=0.94
->Validation phase
  Loss=0.6428
  Accuracy=0.95
-----------------------------------
Epoch 5/8
->Training phase
  Loss=0.1715
  Accuracy=0.98
->Validation phase
  Loss=0.5941
  Accuracy=0.96
-----------------------------------
Epoch 6/8
->Training phase
  Loss=0.0923
  Accuracy=0.98
->Validation phase
  Loss=0.8930
  Accuracy=0.94
-----------------------------------
Epoch 7/8
->Training phase
  Los

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.10 - lr:0.0001 - bs:32 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  lr value 0.0001
  Loss=3.3598
  Accuracy=0.81
->Validation phase
  Loss=0.6297
  Accuracy=0.92
-----------------------------------
Epoch 2/8
->Training phase
  lr value 0.0001
  Loss=0.5268
  Accuracy=0.93
->Validation phase
  Loss=0.8889
  Accuracy=0.92
-----------------------------------
Epoch 3/8
->Training phase
  lr value 5e-05
  Loss=0.1634
  Accuracy=0.97
->Validation phase
  Loss=0.4114
  Accuracy=0.95
-----------------------------------
Epoch 4/8
->Training phase
  lr value 5e-05
  Loss=0.0934
  Accuracy=0.98
->Validation phase
  Loss=0.4601
  Accuracy=0.95
-----------------------------------
Epoch 5/8
->Training phase
  lr value 2.5e-05
  Loss=0.0129
  Accuracy=1.0
->Validation phase
  Loss=0.3573
  Accuracy=0.96
-----------------------------------
Epoch 6/8
->Training phase
  lr value 2.5e-05
  Loss=0.0035
  Accuracy=1.0
->Valida

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.11 - lr:0.0001 - bs:64 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  Loss=6.2268
  Accuracy=0.73
->Validation phase
  Loss=0.5116
  Accuracy=0.93
-----------------------------------
Epoch 2/8
->Training phase
  Loss=0.2968
  Accuracy=0.95
->Validation phase
  Loss=0.4170
  Accuracy=0.93
-----------------------------------
Epoch 3/8
->Training phase
  Loss=0.1416
  Accuracy=0.96
->Validation phase
  Loss=0.3039
  Accuracy=0.95
-----------------------------------
Epoch 4/8
->Training phase
  Loss=0.1544
  Accuracy=0.97
->Validation phase
  Loss=0.4355
  Accuracy=0.93
-----------------------------------
Epoch 5/8
->Training phase
  Loss=0.1060
  Accuracy=0.97
->Validation phase
  Loss=0.3310
  Accuracy=0.95
-----------------------------------
Epoch 6/8
->Training phase
  Loss=0.0472
  Accuracy=0.99
->Validation phase
  Loss=0.4163
  Accuracy=0.94
-----------------------------------
Epoch 7/8
->Training phase
  L

...

Model no.14 - lr:0.0001 - bs:128 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  lr value 0.0001
  Loss=6.7572
  Accuracy=0.63
->Validation phase
  Loss=0.6721
  Accuracy=0.9
-----------------------------------
Epoch 2/8
->Training phase
  lr value 0.0001
  Loss=0.3183
  Accuracy=0.94
->Validation phase
  Loss=0.3416
  Accuracy=0.94
-----------------------------------
Epoch 3/8
->Training phase
  lr value 5e-05
  Loss=0.1368
  Accuracy=0.97
->Validation phase
  Loss=0.3610
  Accuracy=0.94
-----------------------------------
Epoch 4/8
->Training phase
  lr value 5e-05
  Loss=0.0910
  Accuracy=0.98
->Validation phase
  Loss=0.2626
  Accuracy=0.95
-----------------------------------
Epoch 5/8
->Training phase
  lr value 2.5e-05
  Loss=0.0478
  Accuracy=0.99
->Validation phase
  Loss=0.2442
  Accuracy=0.95
-----------------------------------
Epoch 6/8
->Training phase
  lr value 2.5e-05
  Loss=0.0379
  Accuracy=0.99
->Vali

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.15 - lr:0.0001 - bs:256 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  Loss=21.2239
  Accuracy=0.34
->Validation phase
  Loss=8.2429
  Accuracy=0.53
-----------------------------------
Epoch 2/8
->Training phase
  Loss=2.1716
  Accuracy=0.79
->Validation phase
  Loss=0.7269
  Accuracy=0.9
-----------------------------------
Epoch 3/8
->Training phase
  Loss=0.4773
  Accuracy=0.93
->Validation phase
  Loss=0.4543
  Accuracy=0.94
-----------------------------------
Epoch 4/8
->Training phase
  Loss=0.2649
  Accuracy=0.95
->Validation phase
  Loss=0.3989
  Accuracy=0.94
-----------------------------------
Epoch 5/8
->Training phase
  Loss=0.1738
  Accuracy=0.96
->Validation phase
  Loss=0.3261
  Accuracy=0.95
-----------------------------------
Epoch 6/8
->Training phase
  Loss=0.1266
  Accuracy=0.97
->Validation phase
  Loss=0.3027
  Accuracy=0.95
-----------------------------------
Epoch 7/8
->Training phase
  

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.16 - lr:0.0001 - bs:256 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/8
->Training phase
  lr value 0.0001
  Loss=13.3121
  Accuracy=0.43
->Validation phase
  Loss=3.4475
  Accuracy=0.7
-----------------------------------
Epoch 2/8
->Training phase
  lr value 0.0001
  Loss=1.0712
  Accuracy=0.86
->Validation phase
  Loss=0.6082
  Accuracy=0.91
-----------------------------------
Epoch 3/8
->Training phase
  lr value 5e-05
  Loss=0.3220
  Accuracy=0.94
->Validation phase
  Loss=0.3772
  Accuracy=0.94
-----------------------------------
Epoch 4/8
->Training phase
  lr value 5e-05
  Loss=0.2198
  Accuracy=0.95
->Validation phase
  Loss=0.3505
  Accuracy=0.94
-----------------------------------
Epoch 5/8
->Training phase
  lr value 2.5e-05
  Loss=0.1686
  Accuracy=0.96
->Validation phase
  Loss=0.3294
  Accuracy=0.94
-----------------------------------
Epoch 6/8
->Training phase
  lr value 2.5e-05
  Loss=0.1564
  Accuracy=0.97
->Val

In [ ]:
with open(drive_path + 'imagenette_haas_scores_loop3.pickle', 'wb') as f: pickle.dump(haas_scores, f)
with open(drive_path + 'imagenette_balanced_accuracies_loop3.pickle', 'wb') as f: pickle.dump(balanced_accuracies, f)
with open(drive_path + 'imagenette_weighted_f1_scores_loop3.pickle', 'wb') as f: pickle.dump(weighted_f1_scores, f)
with open(drive_path + 'imagenette_mean_roc_auc_scores_loop3.pickle', 'wb') as f: pickle.dump(mean_roc_auc_scores, f)

## **Put all Loop pickles together**

In [ ]:
# LOOP1
with open(drive_path + "imagenette_haas_scores_loop1.pickle", "rb") as f: haas_1 = pickle.load(f)
with open(drive_path + "imagenette_balanced_accuracies_loop1.pickle", "rb") as f: balanced_acc_1 = pickle.load(f)
with open(drive_path + "imagenette_weighted_f1_scores_loop1.pickle", "rb") as f: wf1_1 = pickle.load(f)
with open(drive_path + "imagenette_mean_roc_auc_scores_loop1.pickle", "rb") as f: mean_roc_1 = pickle.load(f)

# LOOP2
with open(drive_path + "imagenette_haas_scores_loop2.pickle", "rb") as f: haas_2 = pickle.load(f)
with open(drive_path + "imagenette_balanced_accuracies_loop2.pickle", "rb") as f: balanced_acc_2 = pickle.load(f)
with open(drive_path + "imagenette_weighted_f1_scores_loop2.pickle", "rb") as f: wf1_2 = pickle.load(f)
with open(drive_path + "imagenette_mean_roc_auc_scores_loop2.pickle", "rb") as f: mean_roc_2 = pickle.load(f)

# LOOP3
with open(drive_path + "imagenette_haas_scores_loop3.pickle", "rb") as f: haas_3 = pickle.load(f)
with open(drive_path + "imagenette_balanced_accuracies_loop3.pickle", "rb") as f: balanced_acc_3 = pickle.load(f)
with open(drive_path + "imagenette_weighted_f1_scores_loop3.pickle", "rb") as f: wf1_3 = pickle.load(f)
with open(drive_path + "imagenette_mean_roc_auc_scores_loop3.pickle", "rb") as f: mean_roc_3 = pickle.load(f)

In [ ]:
haas = haas_1 + haas_2 + haas_3
balanced_acc = balanced_acc_1 + balanced_acc_2 + balanced_acc_3
wf1 = wf1_1 + wf1_2 + wf1_3
mean_roc = mean_roc_1 + mean_roc_2 + mean_roc_3

In [ ]:
with open(drive_path + 'imagenette_haas_scores.pickle', 'wb') as f: pickle.dump(haas, f)
with open(drive_path + 'imagenette_balanced_accuracies.pickle', 'wb') as f: pickle.dump(balanced_acc, f)
with open(drive_path + 'imagenette_weighted_f1_scores.pickle', 'wb') as f: pickle.dump(wf1, f)
with open(drive_path + 'imagenette_mean_roc_auc_scores.pickle', 'wb') as f: pickle.dump(mean_roc, f)

## **Summary**

In [ ]:
import pickle

drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/Imagenette/'

with open(drive_path + "imagenette_mean_roc_auc_scores.pickle", "rb") as f: auc_scores = pickle.load(f)
with open(drive_path + "imagenette_weighted_f1_scores.pickle", "rb") as f: f1_scores = pickle.load(f)
with open(drive_path + "imagenette_haas_scores.pickle", "rb") as f: haas_scores = pickle.load(f)
with open(drive_path + "imagenette_balanced_accuracies.pickle", "rb") as f: accuracies = pickle.load(f)

In [ ]:
import pandas as pd

n = [i for i in range(1,17)]
models = ['vgg19_no{}'.format(i) for i in n]

dictionary = {'Model':models,
              'HAAS score':haas_scores,
              'Mean AUC score':[round(score,3) for score in auc_scores],
              'Accuracy':[round(score,3) for score in accuracies],
              'Macro F1':[round(score,3) for score in f1_scores]}

df = pd.DataFrame.from_dict(dictionary)
df.style.set_properties(**{'text-align': 'center'})
df

,Model,HAAS score,Mean AUC score,Accuracy,Macro F1
0,vgg19_no1,0.994,0.999,0.957,0.959
1,vgg19_no2,1.002,0.995,0.919,0.919
2,vgg19_no3,0.993,0.992,0.898,0.898
3,vgg19_no4,0.994,0.982,0.844,0.844
4,vgg19_no5,0.996,0.985,0.858,0.858
5,vgg19_no6,0.995,0.964,0.761,0.760
6,vgg19_no7,0.995,0.972,0.795,0.791
7,vgg19_no8,0.986,0.889,0.556,0.551
8,vgg19_no9,0.997,0.996,0.900,0.895
9,vgg19_no10,0.994,0.999,0.956,0.956
